# Models

The following cells set up database tables for inserting and querying our eviction data.

In [1]:
from sqlalchemy import create_engine, UniqueConstraint
engine = create_engine('sqlite:///:memory:', echo=True)

In [2]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String, ForeignKey, Table, Text
from sqlalchemy.orm import relationship

In [3]:
class District(Base):
    __tablename__ = 'districts'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    
    attorneys = relationship('Attorney', back_populates='district')
    plantiffs = relationship('Plantiff', back_populates='district')
    defendants = relationship('Defendant', back_populates='district')
    judges = relationship('Judge', back_populates='district')
    courtrooms = relationship('Courtroom', back_populates='district')
    
    def __repr__(self):
        return "<District(name='%s')>" % (self.name)

In [4]:
detainer_warrant_defendants = Table(
    'detainer_warrant_defendants', 
    Base.metadata,
    Column('detainer_warrant_docket_id', ForeignKey('detainer_warrants.docket_id'), primary_key=True),
    Column('defendant_id', ForeignKey('defendants.id'), primary_key=True)
)

In [5]:
class Defendant(Base):
    __tablename__ = 'defendants'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    phone = Column(String)
    address = Column(String, nullable=False)
    
    district_id = Column(Integer, ForeignKey('districts.id'), nullable=False)
    
    UniqueConstraint('name', 'district_id')
    
    district = relationship('District', back_populates='defendants')
    detainer_warrants = relationship('DetainerWarrant',
                                     secondary=detainer_warrant_defendants,
                                     back_populates='defendants'
                                    )

    def __repr__(self):
        return "<Defendant(name='%s', phone='%s', address='%s')>" % (self.name, self.phone, self.address)

In [6]:
class Attorney(Base):
    __tablename__ = 'attorneys'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    district_id = Column(Integer, ForeignKey('districts.id'), nullable=False)
    
    UniqueConstraint('name', 'district_id')
    
    district = relationship('District', back_populates='attorneys')
    plantiff_clients = relationship('Plantiff', back_populates='attorney')
    
    def __repr__(self):
        return "<Attorney(name='%s', district_id='%s')>" % (self.name, self.district_id)

In [7]:
class Courtroom(Base):
    __tablename__ = 'courtrooms'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    district_id = Column(Integer, ForeignKey('districts.id'), nullable=False)
    
    UniqueConstraint('name', 'district_id')
    
    district = relationship('District', back_populates='courtrooms')
    cases = relationship('DetainerWarrant', back_populates='courtroom')
    
    def __repr__(self):
        return "<Courtroom(name='%s')>" % (self.name)

In [8]:
class Plantiff(Base):
    __tablename__ = 'plantiffs'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    attorney_id = Column(Integer, ForeignKey('attorneys.id'))
    district_id = Column(Integer, ForeignKey('districts.id'), nullable=False)
    
    UniqueConstraint('name', 'district_id')
    
    district = relationship('District', back_populates='plantiffs')
    attorney = relationship('Attorney', back_populates='plantiff_clients')
    detainer_warrants = relationship('DetainerWarrant', back_populates='plantiff')
    
    def __repr__(self):
        return "<Plantiff(name='%s', attorney_id='%s', district_id='%s')>" % (self.name, self.attorney_id, self.district_id)

In [9]:
class Judge(Base):
    __tablename__ = "judges"
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    district_id = Column(Integer, ForeignKey('districts.id'), nullable=False)
    
    UniqueConstraint('name', 'district_id')
    
    district = relationship('District', back_populates='judges')
    cases = relationship('DetainerWarrant', back_populates='presiding_judge')
    
    def __repr__(self):
        return "<Judge(name='%s')>" % (self.name)

In [10]:
class DetainerWarrant(Base):
    __tablename__ = 'detainer_warrants'
    docket_id = Column(String, primary_key=True)
    file_date = Column(String, nullable=False)
    status = Column(Integer, nullable=False) # union?
    plantiff_id = Column(Integer, ForeignKey('plantiffs.id'))
    court_date = Column(String, nullable=False) # date
    courtroom_id = Column(Integer, ForeignKey('courtrooms.id'), nullable=False)
    presiding_judge_id = Column(Integer, ForeignKey('judges.id'))
    amount_claimed = Column(String) # USD
    amount_claimed_category = Column(Integer, nullable=False) # enum (POSS | FEES | BOTH | NA)
    judgement = Column(Integer)
    judgement_notes = Column(String)
    
    plantiff = relationship('Plantiff', back_populates='detainer_warrants')
    courtroom = relationship('Courtroom', back_populates='cases')
    presiding_judge = relationship('Judge', back_populates='cases')
    
    defendants = relationship('Defendant',
                              secondary=detainer_warrant_defendants,
                              back_populates='detainer_warrants'
                             )
    def __repr__(self):
        return "<DetainerWarrant(docket_id='%s', file_date='%s')>" % (self.docket_id, self.file_date)


In [11]:
Base.metadata.create_all(engine)

2021-02-23 06:02:22,039 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-02-23 06:02:22,041 INFO sqlalchemy.engine.base.Engine ()
2021-02-23 06:02:22,042 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-02-23 06:02:22,043 INFO sqlalchemy.engine.base.Engine ()
2021-02-23 06:02:22,044 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("districts")
2021-02-23 06:02:22,045 INFO sqlalchemy.engine.base.Engine ()
2021-02-23 06:02:22,047 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("districts")
2021-02-23 06:02:22,049 INFO sqlalchemy.engine.base.Engine ()
2021-02-23 06:02:22,050 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("detainer_warrant_defendants")
2021-02-23 06:02:22,051 INFO sqlalchemy.engine.base.Engine ()
2021-02-23 06:02:22,052 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("detainer_warrant_defendants")
2021-02-23 06:02:22,053 INFO sqlalch

# Spreadsheet Data

We'll be grabbing spreadsheet data from Google here, using the gspread library.

In [12]:
import gspread
gc = gspread.service_account()
sh = gc.open('detainer-warrants_15-02-2020')
ws = sh.worksheet("All Detainer Warrants")

In [13]:
ws.row_values(1)

['Docket #',
 'Order #',
 'File Date',
 'Status',
 'Month',
 'Year',
 'Plantiff',
 'Pltf. Attorney',
 'Court Date',
 'Courtroom',
 'Presiding Judge',
 'Amt Claimed ($)',
 'Amount Claimed (CATEGORY)',
 'Amount Claimed (NON-$)',
 'Defendant(s)',
 'Defendant Address',
 'Def #1 Name',
 'Def #1 Phone',
 'Def #2 Name',
 'Def #2 Phone',
 'Def #3 Name',
 'Def #3 Phone',
 'Judgement']

In [14]:
warrant = ws.row_values(3)

In [15]:
from sqlalchemy.orm import sessionmaker

In [16]:
Session = sessionmaker(bind=engine)
Session.configure(bind=engine)
session = Session()

In [17]:
def init_status(warrant):
    statuses = {
        'CLOSED': 0,
        'PENDING': 1
    }
    return statuses[warrant[3].upper()]

In [18]:
def init_amount_claimed_category(warrant):
    categories = {
        'POSS': 0,
        'FEES': 1,
        'BOTH': 2,
        'N/A': 3,
        '': 4
    }
    return categories[warrant[12].upper()]

In [19]:
def init_judgement(warrant):
    judgements = {} # TODO: work on this
    return judgements[warrant[20]]

In [20]:
from sqlalchemy.sql import ClauseElement

def get_or_create(session, model, defaults=None, **kwargs):
    instance = session.query(model).filter_by(**kwargs).one_or_none()
    if instance:
        return instance, False
    else:
        params = {k: v for k, v in kwargs.items() if not isinstance(v, ClauseElement)}
        params.update(defaults or {})
        instance = model(**params)
        try:
            session.add(instance)
            session.commit()
        except Exception:  # The actual exception depends on the specific database so we catch all exceptions. This is similar to the official documentation: https://docs.sqlalchemy.org/en/latest/orm/session_transaction.html
            session.rollback()
            instance = session.query(model).filter_by(**kwargs).one()
            return instance, False
        else:
            return instance, True


In [21]:
district = District(name="Davidson County")

session.add(district)
session.commit()

2021-02-23 06:02:24,884 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-23 06:02:24,886 INFO sqlalchemy.engine.base.Engine INSERT INTO districts (name) VALUES (?)
2021-02-23 06:02:24,887 INFO sqlalchemy.engine.base.Engine ('Davidson County',)
2021-02-23 06:02:24,890 INFO sqlalchemy.engine.base.Engine COMMIT


In [22]:
defaults = {'district': district}

def detainer_warrant(warrant):
    docket_id = warrant[0]
    file_date = warrant[2]
    status = init_status(warrant)
    attorney, _ = get_or_create(session, Attorney, name=warrant[7], defaults=defaults)
    plantiff, _ = get_or_create(session, Plantiff, name=warrant[6], attorney=attorney, defaults=defaults)
    court_date = warrant[8]
    courtroom, _ = get_or_create(session, Courtroom, name=warrant[9], defaults=defaults)
    presiding_judge, _ = get_or_create(session, Judge, name=warrant[10], defaults=defaults)
    amount_claimed = warrant[11]
    amount_claimed_category = init_amount_claimed_category(warrant)
    defendant, _ = get_or_create(session, Defendant, address=warrant[15], name=warrant[14], phone=warrant[16], defaults=defaults)
    
    return DetainerWarrant(
        docket_id=docket_id,
        file_date=file_date,
        status=status,
        plantiff=plantiff,
        court_date=court_date,
        courtroom=courtroom,
        presiding_judge=presiding_judge,
        amount_claimed=amount_claimed,
        amount_claimed_category=amount_claimed_category,
        defendants=[defendant]
        )

In [23]:
NUM_WARRANTS_TO_INSERT = 5 # insert just a bit of data to play with

for warrant in ws.get_all_values()[1:NUM_WARRANTS_TO_INSERT]:
    session.add(detainer_warrant(warrant))

session.commit()

2021-02-23 06:02:25,910 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-23 06:02:25,911 INFO sqlalchemy.engine.base.Engine SELECT attorneys.id AS attorneys_id, attorneys.name AS attorneys_name, attorneys.district_id AS attorneys_district_id 
FROM attorneys 
WHERE attorneys.name = ?
2021-02-23 06:02:25,912 INFO sqlalchemy.engine.base.Engine ('PRS',)
2021-02-23 06:02:25,914 INFO sqlalchemy.engine.base.Engine SELECT districts.id AS districts_id, districts.name AS districts_name 
FROM districts 
WHERE districts.id = ?
2021-02-23 06:02:25,914 INFO sqlalchemy.engine.base.Engine (1,)
2021-02-23 06:02:25,916 INFO sqlalchemy.engine.base.Engine INSERT INTO attorneys (name, district_id) VALUES (?, ?)
2021-02-23 06:02:25,917 INFO sqlalchemy.engine.base.Engine ('PRS', 1)
2021-02-23 06:02:25,918 INFO sqlalchemy.engine.base.Engine COMMIT
2021-02-23 06:02:25,919 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-23 06:02:25,921 INFO sqlalchemy.engine.base.Engine SELECT attorneys.i

## Querying the database

Refer to the SQLAlchemy [official querying documentation](https://docs.sqlalchemy.org/en/13/orm/tutorial.html#querying).
This reference will be invaluable for learning how to work with this data.

Below are some example queries:

In [24]:
session.query(DetainerWarrant).first().defendants

2021-02-23 06:02:26,123 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-23 06:02:26,125 INFO sqlalchemy.engine.base.Engine SELECT detainer_warrants.docket_id AS detainer_warrants_docket_id, detainer_warrants.file_date AS detainer_warrants_file_date, detainer_warrants.status AS detainer_warrants_status, detainer_warrants.plantiff_id AS detainer_warrants_plantiff_id, detainer_warrants.court_date AS detainer_warrants_court_date, detainer_warrants.courtroom_id AS detainer_warrants_courtroom_id, detainer_warrants.presiding_judge_id AS detainer_warrants_presiding_judge_id, detainer_warrants.amount_claimed AS detainer_warrants_amount_claimed, detainer_warrants.amount_claimed_category AS detainer_warrants_amount_claimed_category, detainer_warrants.judgement AS detainer_warrants_judgement, detainer_warrants.judgement_notes AS detainer_warrants_judgement_notes 
FROM detainer_warrants
 LIMIT ? OFFSET ?
2021-02-23 06:02:26,126 INFO sqlalchemy.engine.base.Engine (1, 0)
2021-02-23 06:02:

[<Defendant(name='CROWE,JARED', phone='JARED CROWE', address='741 LONGHUNTER COURT 37217')>]

In [25]:
session.query(Attorney).first().plantiff_clients

2021-02-23 06:02:26,139 INFO sqlalchemy.engine.base.Engine SELECT attorneys.id AS attorneys_id, attorneys.name AS attorneys_name, attorneys.district_id AS attorneys_district_id 
FROM attorneys
 LIMIT ? OFFSET ?
2021-02-23 06:02:26,140 INFO sqlalchemy.engine.base.Engine (1, 0)
2021-02-23 06:02:26,143 INFO sqlalchemy.engine.base.Engine SELECT plantiffs.id AS plantiffs_id, plantiffs.name AS plantiffs_name, plantiffs.attorney_id AS plantiffs_attorney_id, plantiffs.district_id AS plantiffs_district_id 
FROM plantiffs 
WHERE ? = plantiffs.attorney_id
2021-02-23 06:02:26,144 INFO sqlalchemy.engine.base.Engine (1,)


[<Plantiff(name='DADE,SONDRA A', attorney_id='1', district_id='1')>,
 <Plantiff(name='BATTLE,PATRICIA', attorney_id='1', district_id='1')>,
 <Plantiff(name='DELGADO,PABLO', attorney_id='1', district_id='1')>,
 <Plantiff(name='JERNIGAN,JERRY D', attorney_id='1', district_id='1')>]